In [97]:
import numpy as np
import pandas as pd
from skbio.stats import subsample_counts
from tqdm import tqdm

import math
from statistics import mean 
from scipy import stats
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt

# GOAL: make rarefaction curves for finer taxonomic resolution
### I previously did this for phylum in the folder "s7_diverity/Nycodenz_vs_Oil"

## Method: Split the OTU tab up by taxa groups and then add a "dummy" otu to to represent the rest of the community

In [7]:
wrk="/workdir/users/pd378/oilPCR/all_16s_for_paper/s7_diversity/"
otu_tab=pd.read_csv(wrk+"Nycodenz_vs_Oil/otu_tabs/combined_reps_with_metadata.txt",sep='\t')

grouping_list=[]
for index, row in otu_tab.iterrows():
    if row['treatment']=="nLY":
        grouping_list.append("Lysozyme -")
    elif row['treatment']=="yLY":
        grouping_list.append("Lysozyme +")
    elif row['group']=="Nycodenz":
        grouping_list.append("Nycodenz")
otu_tab["group"]=grouping_list
otu_tab=otu_tab[otu_tab["Donor"]!="UH02"]

otu_tab.head(2)

,Unnamed: 0,Donor,exp,group,sub_group,treatment,sequence_group,otu1,otu10,otu100,...,otu988,otu99,otu990,otu991,otu993,otu995,otu996,otu997,otu998,otu999
0,0,UC02,HtDnLy,Lysozyme -,nDN,nLY,3,0.0,0.0,530.0,...,0.0,428.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,UC02,HtDnLy,Lysozyme +,nDN,yLY,3,5.0,1.0,489.0,...,5.0,445.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


### Split the tabl up by donor and filter empty otus

In [8]:
def filter_zero_otus(filter_df, min_detect, threshold):
    detect_list_out=[]

    this_df=filter_df.copy()
    this_df=this_df[[col for col in this_df.columns if col.startswith('otu')]].T
    #    min_detect=1*len(this_df.columns)
    detect_list=[]
    for row in this_df.iterrows():
        total=0
        for x in row[1].to_list():
            if x >= threshold:
                total=total+1
        detect_list.append(total)


    this_df['detect']=detect_list
    this_df=this_df[this_df['detect']>=min_detect]
    this_df=this_df.drop(['detect'], axis=1)
    return this_df.T


#Split by donor and remove zeros
payload=[]
for donor, group in otu_tab.groupby("Donor"):
    meta_cols=[col for col in group.columns if not col.startswith('otu')]
    df_filtered=filter_zero_otus(group,1,1)
    print(df_filtered.shape)
    df_filtered = pd.concat([group[meta_cols],df_filtered], axis=1)   
    payload.append(df_filtered.drop(["Unnamed: 0"],axis=1))
    print(df_filtered.drop(["Unnamed: 0"],axis=1).shape)

#shuffle
x=payload[1]
payload[1]=payload[0]
payload[0]=x

(12, 951)
(12, 957)
(14, 337)
(14, 343)


### Group the tabs by taxa before adding a new otu to represent the rest of the community
### then prepare for generating the rarefaction curves 

In [189]:
taxa_df=pd.read_csv(wrk+"Nycodenz_vs_Oil/all_taxonomy_split.txt",sep='\t').drop(["Unnamed: 0"], axis=1).set_index("featureid")
taxa_df.head(100)
taxa_df=taxa_df.replace("_","-",regex=True)
taxa_df

,taxonomy,kingdom,phylum,class,order,family,genus,species
featureid,,,,,,,,
otu1,Bacteria(100);Bacteroidetes(100);Bacteroidia(1...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,NaN
otu2,Bacteria(100);Bacteroidetes(100);Bacteroidia(1...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Prevotellaceae,Prevotella-9,NaN
otu3,Bacteria(100);Firmicutes(100);Negativicutes(10...,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Veillonellaceae,Dialister,NaN
otu4,Bacteria(100);Bacteroidetes(100);Bacteroidia(1...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,NaN
otu5,Bacteria(99);Bacteroidetes(99);Bacteroidia(99)...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Rikenellaceae,Rikenellaceae-RC9-gut-group,NaN
...,...,...,...,...,...,...,...,...
otu1841,Bacteria(100);Bacteroidetes(100);Bacteroidia(1...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,NaN
otu1842,Bacteria(100);Firmicutes(82);Clostridia(75);Cl...,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Lactonifactor,NaN
otu1843,Bacteria(100);Bacteroidetes(100);Bacteroidia(1...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Barnesiellaceae,Barnesiella,NaN


In [180]:
taxa_df.columns[2:]

Index(['phylum', 'class', 'order', 'family', 'genus', 'species'], dtype='object')

In [220]:
donor_names=["human","chicken"]
summary_data=[]

otu_threshold=20
for i in range(2):
    df=payload[i].copy()
    o_cols=df[[col for col in df.columns if col.startswith('otu')]].T

    col_names=o_cols.columns
    o_cols=pd.merge(taxa_df[["phylum","class","order","family","genus"]], o_cols, left_index=True, right_index=True, how="right")
    o_cols.head(2)
    
    sample_totals=o_cols[col_names].sum(axis=0)
    
    #make the phylum_tabs
    for taxa, group in o_cols.groupby("phylum"):
        group2=group.drop(["phylum","class","order","family","genus"],axis=1)
        group2=group2.T
        phy_sum=group2.sum(axis=1)
        group2["others"]=sample_totals-phy_sum
        group2=group2.T
        group2.index.name="OTU ID#"
        summary_data.append([donor_names[i],"phylum",taxa,len(group2.index)])
        #only bother with taxa that have enough OTUs
        if len(group2.index) > otu_threshold:
            group2.to_csv(wrk+"taxa_specific_rarefaction/all_split_tabs/"+donor_names[i]+"_phylum_"+taxa+"_na_na_na_na_tab.txt",sep='\t')
            
    #class tabs
    for taxa, group in o_cols.groupby(["phylum","class"]):
        group2=group.drop(["phylum","class","order","family","genus"],axis=1)
        group2=group2.T
        phy_sum=group2.sum(axis=1)
        group2["others"]=sample_totals-phy_sum
        group2=group2.T
        group2.index.name="OTU ID#"
        summary_data.append([donor_names[i],"class",taxa,len(group2.index)])
        taxb=[]
        if len(group2.index) > otu_threshold:
            group2.to_csv(wrk+"taxa_specific_rarefaction/all_split_tabs/"+donor_names[i]+"_class_"+taxa[0]+"_"+taxa[1]+"_na_na_na_tab.txt",sep='\t')
            
       
    #order tabs
    for taxa, group in o_cols.groupby(["phylum","class","order"]):
        group2=group.drop(["phylum","class","order","family","genus"],axis=1)
        group2=group2.T
        phy_sum=group2.sum(axis=1)
        group2["others"]=sample_totals-phy_sum
        group2=group2.T
        group2.index.name="OTU ID#"
        summary_data.append([donor_names[i],"order",taxa,len(group2.index)])
        if len(group2.index) > otu_threshold:
            group2.to_csv(wrk+"taxa_specific_rarefaction/all_split_tabs/"+donor_names[i]+"_order_"+taxa[0]+"_"+taxa[1]+"_"+taxa[2]+"_na_na_tab.txt",sep='\t')
            
    #family tabs        
    for taxa, group in o_cols.groupby(["phylum","class","order","family"]):
        group2=group.drop(["phylum","class","order","family","genus"],axis=1)
        group2=group2.T
        phy_sum=group2.sum(axis=1)
        group2["others"]=sample_totals-phy_sum
        group2=group2.T
        group2.index.name="OTU ID#"
        summary_data.append([donor_names[i],"family",taxa,len(group2.index)])
        if len(group2.index) > otu_threshold:
            group2.to_csv(wrk+"taxa_specific_rarefaction/all_split_tabs/"+donor_names[i]+"_family_"+taxa[0]+"_"+taxa[1]+"_"+taxa[2]+"_"+taxa[3]+"_na_tab.txt",sep='\t')
            
    #genus tabs        
    for taxa, group in o_cols.groupby(["phylum","class","order","family","genus"]):
        group2=group.drop(["phylum","class","order","family","genus"],axis=1)
        group2=group2.T
        phy_sum=group2.sum(axis=1)
        group2["others"]=sample_totals-phy_sum
        group2=group2.T
        group2.index.name="OTU ID#"
        summary_data.append([donor_names[i],"genus",taxa,len(group2.index)])
        if len(group2.index) > otu_threshold:
            group2.to_csv(wrk+"taxa_specific_rarefaction/all_split_tabs/"+donor_names[i]+"_genus_"+taxa[0]+"_"+taxa[1]+"_"+taxa[2]+"_"+taxa[3]+"_"+taxa[4]+"_tab.txt",sep='\t')
            
summm=pd.DataFrame(summary_data, columns=["donor","taxa","full_taxa","otu_count"])
summm.to_csv("summary_of_split_tabs.txt",sep='\t')

#  Generate the rarefaction using "skbio.stats.subsample_counts"
### first reload the files that werre exported

In [221]:
human_files=glob("/workdir/users/pd378/oilPCR/all_16s_for_paper/s7_diversity/taxa_specific_rarefaction/all_split_tabs/human*split_tab.txt")
chicken_files=glob(wrk+"taxa_specific_rarefaction/all_split_tabs/chicken*split_tab.txt")
all_files=glob(wrk+"taxa_specific_rarefaction/all_split_tabs/*tab.txt")

In [222]:
all_files[0].split('/')[9].rsplit("_",2)[0]
in_load={}
for file in all_files:
    in_df=pd.read_csv(file, sep='\t')
    the_key=file.split('/')[9].rsplit("_",2)[0]
    in_load[the_key]=in_df

### Next run the random sampling on all files and store in new dictionary

In [223]:
# a funcion to perform farefaction on a single sample vector within an otu table
# returns a single vector as the average of all replicates
# loop this across all samples for full table

#vector: the list of input counts
#start: the minimum value to use 
#step: how many steps between sampling
#replicates: number of replicates to average
def rare_sample(vector, start, step, replicates):
    count_vector=[]
    for i in range(start,sum(vector),step):
        replicate_tests=[]
        for j in range(replicates):    
            sub = subsample_counts(vector, i)
            replicate_tests.append(np.count_nonzero(sub))
        count_vector.append([i, np.mean(replicate_tests)])
    return count_vector

In [225]:
sample_ref_dict={"12":"n_lyse","13":"y_lyse","14":"n_lyse","15":"y_lyse","16":"n_lyse","17":"y_lyse","18":"n_lyse","19":"y_lyse","20":"nyco","21":"nyco","22":"nyco","23":"nyco","24":"nyco","25":"nyco",
            "0":"n_lyse","1":"y_lyse","2":"n_lyse","3":"y_lyse","4":"n_lyse","5":"y_lyse","6":"n_lyse","7":"y_lyse","8":"nyco","9":"nyco","10":"nyco","11":"nyco"}


rare_dict={}
for key, test_tab in tqdm(in_load.items()):
    rare_full=[]
    for i in range(1,len(test_tab.columns)):
        sample_name=test_tab.columns[i]
        test_col=test_tab[sample_name].astype(int)
        #run the rarefaction
        rare_holder=rare_sample(test_col,10,100,5)
        #go through the output, add the sample name to it, and then add to the master list of results
        for item in rare_holder:
            rare_full.append([sample_name,sample_ref_dict[sample_name]]+item)   
        #turn the output into a dataframe and pivot it
        rare_df=pd.DataFrame(rare_full, columns=["sample","treatment","count","OTUs"])
        rare_df=rare_df.pivot(index="count",columns="sample")["OTUs"]
        #store the dataframe and also export to save progress
        rare_dict[key]=rare_df
        rare_df.to_csv("all_rarefaction/"+key+"_rare.txt",sep='\t')
        

100%|██████████| 38/38 [1:34:48<00:00, 149.69s/it]


In [217]:
key

'chicken_order_Proteobacteria_Gammaproteobacteria_Betaproteobacteriales_na'

In [216]:
rare_df=pd.DataFrame(rare_full, columns=["sample","treatment","count","OTUs"])
rare_df=rare_df.pivot(index="count",columns="sample")["OTUs"]
rare_df.head()

sample,0,1,2,3,4,5,6,7,8,9
count,,,,,,,,,,
10,1.2,1.4,1.2,1.4,1.4,1.0,1.2,1.8,1.2,1.6
110,3.0,3.0,4.4,3.0,3.2,2.8,2.8,2.8,2.4,2.8
210,3.8,3.4,3.8,3.4,4.6,3.2,4.0,3.6,4.0,3.4
310,4.0,4.6,4.6,4.4,5.0,4.6,4.4,4.8,5.0,4.6
410,5.0,5.0,4.8,4.8,5.2,4.4,4.4,4.0,4.6,4.8


In [79]:

test_tab=pd.read_csv("phylum_split_tabs/chicken_Firmicutes_split_tab.txt",sep='\t')

#reference for column names and experimental condition
sample_ref_dict={"12":"n_lyse","13":"y_lyse","14":"n_lyse","15":"y_lyse","16":"n_lyse","17":"y_lyse","18":"n_lyse","19":"y_lyse","20":"nyco","21":"nyco","22":"nyco","23":"nyco","24":"nyco","25":"nyco",
            "0":"n_lyse","1":"y_lyse","2":"n_lyse","3":"y_lyse","4":"n_lyse","5":"y_lyse","6":"n_lyse","7":"y_lyse","8":"nyco","9":"nyco","10":"nyco","11":"nyco"}
human_cols=["OTU ID#","n_lyse","y_lyse","n_lyse","y_lyse","n_lyse","y_lyse","n_lyse","y_lyse","nyco","nyco","nyco","nyco","nyco","nyco"]
chicken_cols=["OTU ID#","n_lyse","y_lyse","n_lyse","y_lyse","n_lyse","y_lyse","n_lyse","y_lyse","nyco","nyco","nyco","nyco"]
ch_yes=["1","3","5","7"]
ch_no=["0","2","4","6"]
hu_yes=["13","15","17","19"]
hu_no=["12","14","16","18"]

,OTU ID#,0,1,2,3,4,5,6,7,8,9,10,11
0,otu1000,0.0,4.0,0.0,5.0,0.0,15.0,0.0,15.0,13.0,16.0,9.0,14.0
1,otu1001,72.0,62.0,74.0,59.0,30.0,95.0,36.0,54.0,63.0,42.0,32.0,21.0


### Reload exported tabled to make reusing code easier

In [139]:

rare_full=[]
for i in tqdm(range(1,len(test_tab.columns))):
    sample_name=test_tab.columns[i]
    test_col=test_tab[sample_name].astype(int)
    #run the rarefaction
    rare_holder=rare_sample(test_col,10,100,2)
    #go through the output, add the sample name to it, and then add to the master list of results
    for item in rare_holder:
     #   item.append(sample_name)
    #    item.append(chicken_dict[sample_name])
        rare_full.append([sample_name,chicken_dict[sample_name]]+item)

100%|██████████| 12/12 [00:53<00:00,  4.49s/it]


In [147]:
rare_df=pd.DataFrame(rare_full, columns=["sample","treatment","count","OTUs"])
rare_df=rare_df.pivot(index="count",columns="sample")["OTUs"]
rare_df.head()


sample,0,1,10,11,2,3,4,5,6,7,8,9
count,,,,,,,,,,,,
10,3.5,5.5,6.0,4.0,3.0,3.5,4.5,3.0,2.5,4.0,4.5,4.0
110,26.0,31.0,30.5,32.5,27.0,26.5,25.0,24.5,23.5,28.5,31.5,33.0
210,40.5,58.5,64.0,47.5,40.5,43.5,36.0,45.0,39.5,44.5,64.0,54.5
310,53.0,61.5,70.5,67.5,49.0,55.0,42.5,63.5,57.0,67.0,70.0,68.0
410,60.0,67.0,90.0,79.0,68.0,79.0,58.5,73.5,60.5,78.5,82.5,73.5


In [148]:
ch_yes=["1","3","5","7"]
ch_no=["0","2","4","6"]
rare_df[ch_yes]

sample,1,3,5,7
count,,,,
10,5.5,3.5,3.0,4.0
110,31.0,26.5,24.5,28.5
210,58.5,43.5,45.0,44.5
310,61.5,55.0,63.5,67.0
410,67.0,79.0,73.5,78.5
...,...,...,...,...
120510,NaN,NaN,NaN,NaN
120610,NaN,NaN,NaN,NaN
120710,NaN,NaN,NaN,NaN


In [149]:
#I need to trim all the replicates to the same depth 
ch_yes=["1","3","5","7"]
ch_no=["0","2","4","6"]
y_half=rare_df[ch_yes].dropna()
n_half=rare_df[ch_no].dropna()
combine=pd.concat([y_half,n_half], axis=1)
#combine["sequences per sample"]=df["sequences per sample"][:len(combine.index)]
#truncate_load[key]=combine

In [150]:
combine

sample,1,3,5,7,0,2,4,6
count,,,,,,,,
10,5.5,3.5,3.0,4.0,3.5,3.0,4.5,2.5
110,31.0,26.5,24.5,28.5,26.0,27.0,25.0,23.5
210,58.5,43.5,45.0,44.5,40.5,40.5,36.0,39.5
310,61.5,55.0,63.5,67.0,53.0,49.0,42.5,57.0
410,67.0,79.0,73.5,78.5,60.0,68.0,58.5,60.5
...,...,...,...,...,...,...,...,...
103710,455.5,463.0,465.5,460.0,NaN,NaN,NaN,NaN
103810,456.0,466.5,466.0,459.5,NaN,NaN,NaN,NaN
103910,455.5,465.5,466.0,453.0,NaN,NaN,NaN,NaN


In [64]:
rare_sample(test_tab["0"].astype(int),100,1)

[[100, 22.0],
 [200, 44.0],
 [300, 51.0],
 [400, 54.0],
 [500, 80.0],
 [600, 69.0],
 [700, 93.0],
 [800, 92.0],
 [900, 101.0],
 [1000, 115.0],
 [1100, 114.0],
 [1200, 126.0],
 [1300, 119.0],
 [1400, 127.0],
 [1500, 139.0],
 [1600, 136.0],
 [1700, 133.0],
 [1800, 147.0],
 [1900, 151.0],
 [2000, 159.0],
 [2100, 152.0],
 [2200, 165.0],
 [2300, 173.0],
 [2400, 169.0],
 [2500, 178.0],
 [2600, 178.0],
 [2700, 183.0],
 [2800, 185.0],
 [2900, 182.0],
 [3000, 180.0],
 [3100, 169.0],
 [3200, 183.0],
 [3300, 196.0],
 [3400, 183.0],
 [3500, 201.0],
 [3600, 206.0],
 [3700, 205.0],
 [3800, 204.0],
 [3900, 214.0],
 [4000, 218.0],
 [4100, 207.0],
 [4200, 209.0],
 [4300, 210.0],
 [4400, 215.0],
 [4500, 207.0],
 [4600, 219.0],
 [4700, 219.0],
 [4800, 212.0],
 [4900, 223.0],
 [5000, 230.0],
 [5100, 219.0],
 [5200, 228.0],
 [5300, 222.0],
 [5400, 223.0],
 [5500, 227.0],
 [5600, 233.0],
 [5700, 239.0],
 [5800, 245.0],
 [5900, 249.0],
 [6000, 242.0],
 [6100, 240.0],
 [6200, 239.0],
 [6300, 242.0],
 [6400, 2